Check the large scale galaxy bias as implied by the HOD models we use for each source and lens sample.

In [1]:
# Import modules
% matplotlib inline
import numpy as np; 
import scipy.integrate; 
import scipy.interpolate; 
import matplotlib.pyplot as plt
import pyccl as ccl; 
import shared_functions_wlp_wls as shared; 
import shared_functions_setup as setup

/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/halo_table_cache.py:12: UserWarning: Some of the functionality of the HaloTableCache classrequires h5py to be installed.
  warn("Some of the functionality of the HaloTableCache class"
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/user_supplied_ptcl_catalog.py:13: UserWarning: Most of the functionality of the sim_manager sub-package requires h5py to be installed,
which can be accomplished either with pip or conda
  warn("Most of the functionality of the sim_manager "
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/download_manager.py:39: UserWarning: Some of the functionality of the DownloadManager requires h5py to be installed,
which can be accomplished either with pip or conda
  warn("Some of the functionality of the DownloadManager requires h5py to be installed,\n"
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/cached_halo_catalog.py:15: UserWarning: Most of the functionality of the

In [2]:
# Set the survey
survey = 'SDSS'
if (survey == 'SDSS'):
    import params as pa
elif (survey == 'LSST_DESI'):
    import params_LSST_DESI as pa

In [3]:
#Initialize the cosmology
OmM = pa.OmC_l + pa.OmB_l; OmB=pa.OmB_l; h0 = pa.HH0_l / 100.; sigma80=pa.sigma8_l; n_s0 = pa.n_s_l;
#OmM = 0.25; OmB=0.05; h0 = pa.HH0_l / 100.; sigma80=0.79; n_s0 = pa.n_s_l;

p = ccl.Parameters(Omega_c = OmM-OmB, Omega_b = OmB, h = h0, sigma8=sigma80, n_s=n_s0)
cosmo = ccl.Cosmology(p)

rho_crit = 3. * 10**10 * pa.mperMpc / (8. * np.pi * pa.Gnewt * pa.Msun)  # Msol h^2 / Mpc^3, for use with M in Msol / h (comoving distances)
rho_m = (OmM) * rho_crit # units of Msol h^2 / Mpc^3 (comoving distances)

In [ ]:
# Get the window function of sources x lenses (this is the redshift range we care about)
#(z, dNdz) = shared.window(survey)    

# At the moment get the window function of just lenses
z = np.linspace(pa.zLmin, pa.zLmax, 100)
dNdz = setup.get_dNdzL(z, survey)
    
# Get the halo mass function and halo bias
Mhvec = np.logspace(9.,16,30) # In units Msol / h
HMF = np.zeros((len(Mhvec), len(z)))
#bh = np.zeros((len(Mhvec), len(z)))
for zi in range(0,len(z)):
    HMF[:,zi] = ccl.massfunction.massfunc(cosmo, Mhvec /h0, 1./ (1. + z[zi]), odelta=200.) / h0**3
#HMF= ccl.massfunction.massfunc(cosmo, Mhvec / h0, 1./ (1. + z), odelta=200.) / h0**3

In [ ]:
# Get y(k,M) (the fourier transformed profile)
logkmin = -6; kpts =40000; logkmax = 5; 
kvec_FT = np.logspace(logkmin, logkmax, kpts)
# Actually we will use a downsampled version of this:
k = np.logspace(np.log10(kvec_FT[0]), np.log10(kvec_FT[-1]), 40)
y = shared.gety_ldm(Mhvec, k, survey)

In [ ]:
# Now, we want to convert this to a 1-halo and 2-halo galaxy power spectrum using the various HOD's we use.
# We need Mstarlow for the Zu & Mandelbaum halo model
#tot_nsrc = shared.vol_dens(pa.fsky, pa.N_shapes, survey)
#Mstarlow = shared.get_Mstar_low(survey, tot_nsrc)
#print "Mstarlow=", Mstarlow

In [ ]:
# Get occupation numbers as a function of mass
if (survey == 'SDSS'):
    Ncen_lens = shared.get_Ncen_Reid(Mhvec, survey)  # Reid & Spergel
    Nsat_lens = shared.get_Nsat_Reid(Mhvec, survey)  # Reid & Spergel 
    #Ncen_src = shared.get_Ncen_Zu(Mhvec, Mstarlow, survey)  	# Zu & Mandelbaum 2015
    #Nsat_src_wlens = shared.get_Nsat_Zu(Mhvec, Mstarlow, 'with_lens', survey)  # Zu & Mandelbaum 2015
    #Nsat_src_tot = shared.get_Nsat_Zu(Mhvec, Mstarlow, 'tot', survey)  # Zu & Mandelbaum 2015
elif (survey== 'LSST_DESI'):
    Ncen_lens = shared.get_Ncen_More(Mhvec, survey) # CMASS
    Nsat_lens = shared.get_Nsat_More(Mhvec, survey) # CMASS 
    Nsat_src_wlens = shared.get_Nsat_Zu(Mhvec, Mstarlow, 'with_lens', survey)  # Zu & Mandelbaum 2015
    Nsat_src_tot = shared.get_Nsat_Zu(Mhvec, Mstarlow, 'tot', survey)  # Zu & Mandelbaum 2015
    Ncen_src = shared.get_Ncen_Zu(Mhvec, Mstarlow, survey)  	# Zu & Mandelbaum 2015

# Uncomment the following to plot some checks    
    
#plt.figure(figsize=[5,5])
#plt.semilogx(Mhvec / h0, Ncen_lens, linestyle='--')
#plt.hold(True)
#plt.semilogx(Mhvec/h0, Nsat_lens)
#plt.ylim(0, 7)
#plt.xlim(10**13, 2.5*10**15)
#plt.ylabel('$<N_{cen}(M) N_{sat}(M)>$')
#plt.xlabel('$M/M_\odot$')
#plt.savefig('./plots/Nsat_Reid2008.pdf')   

#interp_Nsat = scipy.interpolate.interp1d(Mhvec/h0, Nsat_lens)

#Mh_morepoints = np.logspace(13, np.log10(2.5*10**15),100)
#Nsat_morepoints = interp_Nsat(Mh_morepoints)
#print zip(Mh_morepoints, Nsat_morepoints)
    
#plt.figure()
#plt.loglog(Mhvec/(pa.HH0_s/100.), Ncen_lens, 'k', label='Lens centrals')
#plt.hold(True)
#plt.loglog(Mhvec/(pa.HH0_s/100.), Nsat_lens, 'm', label='Lens satellites')
#plt.hold(True)
#plt.loglog(Mhvec/(pa.HH0_s/100.), Nsat_src_wlens, 'g', label='Source satellites')
#plt.hold(True)
#plt.axhline(y=1)
#plt.ylim(0.01,1000)
#plt.xlim(10**11, 3*10**15)
#plt.ylabel('$<N(M)>$')
#plt.xlabel('$M/M_\odot$')
#plt.legend(loc='upper left')
#plt.savefig('./plots/N(M).png')""" 
    
#plt.figure()
#plt.semilogx(Mhvec/(pa.HH0_s/100.) , Ncen_lens)
#plt.hold(True)
#plt.semilogx(Mhvec/(pa.HH0_s/100.), Ncen_lens * Nsat_lens)
#plt.ylim(0,7)
#plt.xlim(10**9, 3*10**15)
#plt.ylabel('$<N_{cen}(M) N_{sat}(M)>$')
#plt.xlabel('$M/M_\odot$')
#plt.show()

In [ ]:
#HMF_intz = np.zeros(len(Mhvec))
#for mi in range(0,len(Mhvec)):
#    HMF_intz[mi] = scipy.integrate.simps(HMF[mi, :]*dNdz, z)
    
#dnsatdlogM = np.abs(HMF_intz * 10**9 * Nsat_lens) 

#print "dndsadlogM=", Nsat_lens

#plt.figure()
#plt.loglog(Mhvec / h0, dnsatdlogM, 'bo')
#plt.xlim(2*10**13, 3*10**15)
#plt.ylim(0, 8500)
#plt.show()

In [ ]:
# Combine to get the total occupation at mass M
N_tot_lens= Ncen_lens + Nsat_lens
#N_tot_src = Ncen_src + Nsat_src_tot

# Get satelite fraction integrated over mass
Nsat_int_ofz_lens = np.zeros(len(z))
Ntot_int_ofz_lens = np.zeros(len(z))
#Nsat_int_ofz_src = np.zeros(len(z))
#Ntot_int_ofz_src = np.zeros(len(z))
for zi in range(0,len(z)):
    Nsat_int_ofz_lens[zi] = scipy.integrate.simps(Nsat_lens * HMF[:,zi], np.log10(Mhvec / h0))
    Ntot_int_ofz_lens[zi] = scipy.integrate.simps(N_tot_lens * HMF[:,zi], np.log10(Mhvec / h0))
    #Nsat_int_ofz_src[zi] = scipy.integrate.simps(Nsat_src_tot * HMF[:,zi], np.log10(Mhvec / pa.HH0_s/100.))
    #Ntot_int_ofz_src[zi] = scipy.integrate.simps(N_tot_src * HMF[:,zi], np.log10(Mhvec / pa.HH0_s/100.))
    
Nsat_int_lens = scipy.integrate.simps(Nsat_int_ofz_lens * dNdz, z)
Ntot_int_lens = scipy.integrate.simps(Ntot_int_ofz_lens * dNdz, z)    
#Nsat_int_src = scipy.integrate.simps(Nsat_int_ofz_src * dNdz, z)
#Ntot_int_src = scipy.integrate.simps(Ntot_int_ofz_src * dNdz, z)
#Nsat_int = scipy.integrate.simps(Nsat_lens * HMF, np.log10(Mhvec))
#Ntot_int= scipy.integrate.simps(N_tot_lens * HMF, np.log10(Mhvec))
satfrac_lens = Nsat_int_lens / Ntot_int_lens
#satfrac_lens = Nsat_int / Ntot_int
#satfrac_src = Nsat_int_src / Ntot_int_src
print "sat frac lens=", satfrac_lens #, "src=", satfrac_src
    
# Integrate over the halo mass function to get total number density 
nbar_lens = np.zeros(len(z))
#nbar_sat_lens = np.zeros(len(z))
#nbar_cen_lens = np.zeros(len(z))
#nbar_src = np.zeros(len(z))
#nbar_sat_src = np.zeros(len(z))
#nbar_cen_src = np.zeros(len(z))
for zi in range(0,len(z)):
    nbar_lens[zi] = scipy.integrate.simps(HMF[:,zi] * N_tot_lens, np.log10(Mhvec / h0))
#    nbar_sat_lens[zi]= scipy.integrate.simps(HMF[:,zi] * Nsat_lens, np.log10(Mhvec / (pa.HH0_s/100.)))
#    nbar_cen_lens[zi]= scipy.integrate.simps(HMF[:,zi] * Ncen_lens, np.log10(Mhvec / (pa.HH0_s/100.)))
#    nbar_src[zi] = scipy.integrate.simps(HMF[:,zi] * N_tot_src, np.log10(Mhvec / (pa.HH0_s/100.)))
#    nbar_sat_src[zi]= scipy.integrate.simps(HMF[:,zi] * Nsat_src_tot, np.log10(Mhvec / (pa.HH0_s/100.)))
#    nbar_cen_src[zi]= scipy.integrate.simps(HMF[:,zi] * Ncen_src, np.log10(Mhvec / (pa.HH0_s/100.)))
                                
#nbar_int_lens= scipy.integrate.simps(HMF * N_tot_lens, np.log10(Mhvec)) 
#nbar_sat_int= scipy.integrate.simps(HMF * Nsat, np.log10(Mhvec / ((pa.HH0/100.))))
#nbar_cen_int= scipy.integrate.simps(HMF * Ncen, np.log10(Mhvec / ((pa.HH0/100.))))                                
    
nbar_int_lens = scipy.integrate.simps(nbar_lens *dNdz, z)
#nbar_cen_int_lens = scipy.integrate.simps(nbar_cen_lens * dNdz, z)
#nbar_sat_int_lens = scipy.integrate.simps(nbar_sat_lens*dNdz, z)
#nbar_int_src = scipy.integrate.simps(nbar_src *dNdz, z)
#nbar_cen_int_src = scipy.integrate.simps(nbar_cen_src * dNdz, z)
#nbar_sat_int_src = scipy.integrate.simps(nbar_sat_src*dNdz, z)"""

#print "lens halo bias, centrals=", bcen_int_lens / nbar_cen_int_lens
#print "lens halo bias, satelites =", bsat_int_lens / nbar_sat_int_lens
#print "lens halo bias, all =", btot_int_lens / nbar_int_lens
print "lens nbar int=", nbar_int_lens

#print "src halo bias, centrals=", bcen_int_src / nbar_cen_int_src
#print "src halo bias, satelites =", bsat_int_src / nbar_sat_int_src
#print "src halo bias, all =", btot_int_src / nbar_int_src
#print "src nbar int=", nbar_int_src


In [ ]:
# Let the galaxy-galaxy two-halo term be given by halofit * both biases as found from these HOD

# DON'T FORGET that CCL power spectra are not in little h units. P(k) has units Mpc^3, takes k in units 1/Mpc

Pknonlin = np.zeros((len(k), len(z)))
#Pklin = np.zeros((len(k), len(z)))
for zi in range(0,len(z)):
    # Change k and Pk to little h units
    Pknonlin[:, zi] = ccl.power.nonlin_matter_power(cosmo, k * h0, 1. / (1. + z[zi])) * h0**3
    #Pklin[:, zi] = ccl.power.linear_matter_power(cosmo, k * h0, 1. / (1. + z[zi])) * h0**3
    
Pkgg = pa.bd*pa.bd * Pknonlin
#Pkgg_lin = pa.bd * pa.bd * Pklin

# Integrate over z
P_2h_gg_avgz = np.zeros(len(k))
#P_2h_gg_avgz_lin = np.zeros(len(k))
for ki in range(0,len(k)):
    P_2h_gg_avgz[ki] = scipy.integrate.simps(dNdz * Pkgg[ki,:], z)
    #P_2h_gg_avgz_lin[ki] = scipy.integrate.simps(dNdz * Pkgg_lin[ki,:], z)

In [ ]:
# Construct now the galaxy - galaxy 1-halo term from the HOD:

#NcenL_NsatS = shared.NcenNsat(1, Ncen_lens, Nsat_src_wlens)
#NsatL_NsatS = shared.NsatNsat(1, Nsat_lens, Nsat_src_wlens)

NcenL_NsatL = Ncen_lens * Nsat_lens

#print "Ncen_lens=", Ncen_lens
#print "Nsat_lens=", Nsat_lens

print "NcenL_NsatL=", NcenL_NsatL

# For the term that is satellite x satellite: 
# We're looking at 1-halo here, so once you've picked a satellite as part of your pair, there's one less
# satellite you can pair it with (can't pair it with itself)
# But note also: if Nsat_lens<1, ie if the mean number of satellites in a halof of mass M<1, this term is 0.
# Can't naived do Nsat(Nsat-1) because you'll get negative numbers
NsatL_NsatL = np.zeros(len(Mhvec))
for mi in range(0,len(Mhvec)):
    if (Nsat_lens[mi]>1.0):
        NsatL_NsatL[mi] = Nsat_lens[mi] * (Nsat_lens[mi] -1.)
    else:
        NsatL_NsatL[mi] = 0.
#NsatL_NsatL = Nsat_lens * (Nsat_lens -1.)
print "NsatL_NsatL=", NsatL_NsatL

In [ ]:
Pkgg_1h = np.zeros((len(k), len(z)))
Pkgg_1h_cen_sat = np.zeros((len(k), len(z)))
Pkgg_1h_sat_sat = np.zeros((len(k), len(z)))
for ki in range(0,len(k)):
    for zi in range(0,len(z)):
        #Pkgg_1h[ki, zi] = scipy.integrate.simps( HMF[:, zi] * (NcenL_NsatS * y[ki, :] + NsatL_NsatS * y[ki, :]**2), np.log10(Mhvec / (pa.HH0_s/100.)  )) / (nbar_src[zi] * nbar_lens[zi])
        Pkgg_1h[ki, zi] = scipy.integrate.simps( HMF[:, zi] * (NcenL_NsatL * y[ki, :] + NsatL_NsatL * y[ki, :]**2), np.log10(Mhvec /h0)) / (nbar_lens[zi] * nbar_lens[zi])
        #Pkgg_1h_cen_sat[ki, zi] = scipy.integrate.simps( HMF[:, zi] * (NcenL_NsatL * y[ki, :]), np.log10(Mhvec /h0)) / (nbar_lens[zi] * nbar_lens[zi])
        #Pkgg_1h_sat_sat[ki, zi] = scipy.integrate.simps( HMF[:, zi] * NsatL_NsatL * y[ki, :]**2, np.log10(Mhvec /h0)) / (nbar_lens[zi] * nbar_lens[zi])
 
#print "Pkgg_1h_satsat=", Pkgg_1h_sat_sat

Pgg1h_avgz = np.zeros(len(k))
#Pgg1h_avgz_cs = np.zeros(len(k))
#Pgg1h_avgz_ss = np.zeros(len(k))
for ki in range(0,len(k)):
    Pgg1h_avgz[ki] = scipy.integrate.simps(dNdz * Pkgg_1h[ki,:], z)   
    #Pgg1h_avgz_cs[ki] = scipy.integrate.simps(dNdz * Pkgg_1h_cen_sat[ki,:], z) 
    #Pgg1h_avgz_ss[ki] = scipy.integrate.simps(dNdz * Pkgg_1h_sat_sat[ki,:], z)
    
#print "Pkgg_1h_satsat=", Pkgg_1h_sat_sat

#interp_in_k = scipy.interpolate.interp1d(np.log(k), np.log(Pgg1h_avgz_cs))
#censat_Pgg_klong = np.exp(interp_in_k(np.log(kvec_FT)))

#savePgg = np.column_stack((kvec_FT, censat_Pgg_klong))
#np.savetxt('./txtfiles/halofit_Pk/Pk_1h_cs_SDSS.txt', savePgg)    
    
#interp_in_k = scipy.interpolate.interp1d(np.log(k), np.log(Pgg1h_avgz_ss))
#satsat_Pgg_klong = np.exp(interp_in_k(np.log(kvec_FT)))

#savePgg = np.column_stack((kvec_FT, satsat_Pgg_klong))
#np.savetxt('./txtfiles/halofit_Pk/Pk_1h_ss_SDSS.txt', savePgg)        
    
interp_in_k = scipy.interpolate.interp1d(np.log(k), np.log(Pgg1h_avgz + P_2h_gg_avgz))
full_Pgg_klong = np.exp(interp_in_k(np.log(kvec_FT)))

savePgg = np.column_stack((kvec_FT, full_Pgg_klong))
np.savetxt('./txtfiles/halofit_Pk/Pk_1h2h_SDSS_lens_fixNsat.txt', savePgg)

#interp_in_k = scipy.interpolate.interp1d(np.log(k), np.log(P_2h_gg_avgz))
#twoh_Pgg_klong = np.exp(interp_in_k(np.log(kvec_FT)))

#savePgg = np.column_stack((kvec_FT, twoh_Pgg_klong))
#np.savetxt('./txtfiles/halofit_Pk/Pk_2h_SDSS_lens_fixhunits.txt', savePgg)

#interp_in_k = scipy.interpolate.interp1d(np.log(k), np.log(P_2h_gg_avgz))
#twoh_Pgg_klong = np.exp(interp_in_k(np.log(kvec_FT)))

#savePgg = np.column_stack((kvec_FT, twoh_Pgg_klong))
#np.savetxt('./txtfiles/halofit_Pk/Pk_2h_SDSS_lens_Jan9.txt', savePgg)

#interp_in_k_1h = scipy.interpolate.interp1d(np.log(k), np.log(Pgg1h_avgz))
#oneh_Pgg_klong = np.exp(interp_in_k_1h(np.log(kvec_FT)))
   
#savePgg = np.column_stack((kvec_FT, full_Pgg_klong))
#np.savetxt('./txtfiles/halofit_Pk/Pk_1h2h_SDSS_lens_Jan8.txt', savePgg)

    
#plt.figure()
#plt.loglog(k,Pgg1h_avgz_ss,  'b', label='1halo')
#plt.hold(True)
#plt.loglog(k, P_2h_gg_avgz, 'g', label='2halo')
#plt.hold(True)
#plt.loglog(k, Pgg1h_avgz + P_2h_gg_avgz, 'm', label='1h+2h')
#plt.xlim(0.0001,30)
#plt.ylim(0.01, 10**7)
#plt.ylabel('$P(k)$')
#plt.xlabel('$k$, h/Mpc')
#plt.title('$P(k)$ terms, SDSS')
#plt.legend()
#plt.show()
#plt.savefig('./plots/Pk_SDSS.pdf')
        

In [ ]:
# Load  the combined xi from file after Fourier transforming

(r, xi_tot) = np.loadtxt('./txtfiles/halofit_xi/xi_1h2h_SDSS_lens_fixNsat.txt', unpack=True)
#(r, xi_cs) = np.loadtxt('./txtfiles/halofit_xi/xi_1h_cs_SDSS.txt', unpack=True)
#(r, xi_ss) = np.loadtxt('./txtfiles/halofit_xi/xi_1h_ss_SDSS.txt', unpack=True)

plt.figure(figsize=[10,10])
plt.loglog(r, xi_tot, 'm')
#plt.loglog(r, xi_cs, 'm', label='cen-sat')
#plt.hold(True)
#plt.loglog(r, xi_ss, 'g', label='sat-sat')
plt.ylabel('$\\xi(r)$')
plt.xlabel('$r$, Mpc/h')
plt.xlim(0.01, 100)
plt.ylim(0.001, 10**6)
plt.title('$\\xi(r)$, SDSS')
plt.legend()
plt.show()
#plt.savefig('./plots/xi_SDSS.png')

In [ ]:
# Project:

for ri in range(0,len(r)):
    if xi_tot[ri]<10**(-12):
        xi_tot[ri]=10**(-12)
#for ri in range(0,len(r)):
#    if xi_cs[ri]<10**(-12):
#        xi_cs[ri]=10**(-12)
        
#for ri in range(0,len(r)):
#    if xi_ss[ri]<10**(-12):        
#        xi_ss[ri]=10**(-12)

#interpolate in r:
interp_xi = scipy.interpolate.interp1d(np.log(r), np.log(xi_tot))
#interp_xiss = scipy.interpolate.interp1d(np.log(r), np.log(xi_ss))
#interp_xics = scipy.interpolate.interp1d(np.log(r), np.log(xi_cs))

rp = np.logspace(-2, 2, 100)
Pi = np.logspace(-5, np.log10(80.), 1000) # use the same projection length as Reid & Spergel 2008

logxi_2d = np.zeros((len(rp), len(Pi)))
#logxi_2d_cs = np.zeros((len(rp), len(Pi)))
#logxi_2d_ss = np.zeros((len(rp), len(Pi)))
for ri in range(0,len(rp)):
    for pi in range(0,len(Pi)):
        r_2d = np.sqrt( rp[ri]**2 + Pi[pi]**2 )
        logxi_2d[ri, pi] = interp_xi(np.log(r_2d))
        #logxi_2d_cs[ri,pi] = interp_xics(np.log(r_2d))
        #logxi_2d_ss[ri,pi] = interp_xiss(np.log(r_2d))

wgg = [0]*len(rp)
#wgg_cs = [0] * len(rp)
#wgg_ss = [0] * len(rp)
for ri in range(0,len(rp)):
   # print np.log(np.sqrt(rp[ri]**2 + Pi**2))
    wgg[ri] = 2.*scipy.integrate.simps(np.exp(logxi_2d[ri,:]), Pi)
    #wgg_cs[ri] = 2.*scipy.integrate.simps(np.exp(logxi_2d_cs[ri,:]), Pi)
    #wgg_ss[ri] = 2.*scipy.integrate.simps(np.exp(logxi_2d_ss[ri,:]), Pi)
    
plt.figure()
plt.loglog(rp, np.asarray(wgg), 'm', label='1h+2h')
#plt.loglog(rp, np.asarray(wgg_cs), 'm', label='cen-sat')
#plt.hold(True)
#plt.loglog(rp, np.asarray(wgg_ss), 'k', label='sat-sat')
plt.ylabel('$w_{gg}$')
plt.xlabel('$r_p$, Mpc/h')
plt.title('$w_{gg}$, SDSS')
plt.xlim(0.04, 30)
plt.ylim(8, 7000)
plt.legend()
plt.show()
#plt.savefig('./plots/wgg_SDSS_fixhunits.pdf')

In [ ]:
zip(rp, wgg)

In [ ]:
# wgg points read off from Reid & Spergel 2008 / Masjedi 2006

rp_Masjedi = np.asarray([0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8])
wgg_Masjedi = np.asarray([6700., 5500., 4050., 3700., 3200., 3000., 1350.,1000.,780., 610., 490., 350. ])

In [ ]:
inter_wgg_HOD = scipy.interpolate.interp1d(rp, wgg)

In [ ]:
wgg_HOD_short = inter_wgg_HOD(rp_Masjedi)

In [ ]:
plt.figure()
plt.loglog(rp_Masjedi, wgg_Masjedi, 'mo')
plt.hold(True)
plt.loglog(rp_Masjedi, wgg_HOD_short)
plt.xlim(0.04, 30)
plt.ylim(8, 7000)
plt.show()

In [ ]:
wgg_Masjedi / wgg_HOD_short